# Assignment_3_CS350

<ins>Objective</ins>: An analysis of the fraud detection database, for the possibility of second-party fraud.

<ins>Create</ins>: Jupyter notebook for the analysis, in Python

<ins>Steps</ins>: For list of steps required to complete this assignment please visit Class 20 slides, there are 14 steps.

## Step 4: Write the appropriate notebook cells that will allow you to connect to the Neo4j server and to use Neo4j GDS.

In [ ]:
# Install required libraries
!pip install neo4j graphdatascience

In [ ]:
# Example of checking database connection
from neo4j import GraphDatabase
driver = GraphDatabase.driver("bolt://localhost:7689", auth=("neo4j", "password"))
with driver.session() as session:
    result = session.run("RETURN 1")
    print(result.single())


In [ ]:
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

# Connection details
bolt_uri = "bolt://localhost:7689"  # Change to your Neo4j Bolt URI. This is default
username = "neo4j"                  # Replace with your username. This is default
password = "password"               # Replace with your password. This is mine.

# Connect to Neo4j
driver = GraphDatabase.driver(bolt_uri, auth=(username, password))
gds = GraphDataScience(bolt_uri, auth=(username, password))

# Test connection
try:
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful!' AS message")
        print(result.single()["message"])
except Exception as e:
    print("Connection failed:", e)


## Step 5: Create the SHARED_PII relationship (between clients) and the resulting subgraph.

In [ ]:
# Sample data to create Clients and SHARED_PII relationships
def create_shared_pii_relationships(session):    
    # Create a SHARED_PII relationship between them
    session.run("""
        MATCH ( c:Client )-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ] -> (n) <-
        [ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]- ( d:Client )
        WHERE id(c) < id(d)
        WITH c, d, count(*) AS cnt
        MERGE (c) - [ :SHARED_PII { count: cnt } ] -> (d)
    """)

# Execute the creation of relationships
try:
    with driver.session() as session:
        create_shared_pii_relationships(session)
        print("SHARED_PII relationship created between clients.")
except Exception as e:
    print("Error creating relationship:", e)


## Step 6: Make the in-memory projection of the graph in STEP 5.

In [ ]:
# Create the in-memory projection of the graph
def create_in_memory_projection(gds):
    # Project the graph 'clientClusters' with nodes labeled 'Client' and relationships of type 'SHARED_PII'
    try:
        graph = gds.graph.project(
            "clientClusters",  # Name of the projection
            "Client",         # Node label to include (Client nodes)
            "SHARED_PII"      # Relationship type to include (SHARED_PII relationships)
        )
        print("In-memory projection 'clientClusters' created.")
    except Exception as e:
        print("Error creating in-memory projection:", e)

# Execute the in-memory projection
try:
    with driver.session() as session:
        create_in_memory_projection(gds)
except Exception as e:
    print("Error executing projection:", e)


## Step 7: use the WCC (Weakly Connected Components) algorithm, in stream mode, to identify clusters of Client nodes in the above projection graph;

In [ ]:
import pandas as pd
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

# Connection details
bolt_uri = "bolt://localhost:7689"  # Adjust with your own URI
username = "neo4j" # Adjust to your own username
password = "password" # Adjust to your own password

# Initialize the Neo4j driver
driver = GraphDatabase.driver(bolt_uri, auth=(username, password))

# Initialize GraphDataScience instance
gds = GraphDataScience(bolt_uri, auth=(username, password))

# Run WCC algorithm in stream mode
query = """
CALL gds.wcc.stream(
  'clientClusters', 
  {
    nodeLabels: ['Client'],
    relationshipTypes: ['SHARED_PII'],
    consecutiveIds: true
  }
)
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).id AS clientId, componentId AS clusterId
"""

# Execute the query and convert results to Pandas DataFrame
with driver.session() as session:
    result = session.run(query)
    data = [{"clientId": record["clientId"], "clusterId": record["clusterId"]} for record in result]
    df = pd.DataFrame(data)

# Display the DataFrame
print(df)

## Step 8 mark each client that belongs to a cluster of size at least 2 as possibly (not provably) belonging to a fraud ring;

In [ ]:
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

# Connection details
bolt_uri = "bolt://localhost:7689"  # Adjust with your own URI
username = "neo4j"
password = "password"

# Initialize the Neo4j driver
driver = GraphDatabase.driver(bolt_uri, auth=(username, password))

# Initialize GraphDataScience instance
gds = GraphDataScience(bolt_uri, auth=(username, password))

# Step 8 - Mark clients in clusters with size >= 2
query = """
CALL gds.wcc.stream(
  'clientClusters', 
  {
    nodeLabels: ['Client'],
    relationshipTypes: ['SHARED_PII'],
    consecutiveIds: true
  }
)
YIELD nodeId, componentId
WITH gds.util.asNode(nodeId) AS clientId, componentId AS clusterId
WITH clusterId, collect(clientId.id) AS clients
WITH clusterId, clients, size(clients) AS clusterSize
WHERE clusterSize >= 2
UNWIND clients AS client
MATCH (c:Client) WHERE c.id = client
SET c.secondPartyFraudRing = clusterId
"""

# Execute the query to mark the clients
with driver.session() as session:
    session.run(query)

print("Clients in clusters of size >= 2 have been marked as possible fraudsters.")

## Step 9

In [ ]:
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

# Connection details
bolt_uri = "bolt://localhost:7689"  # Adjust with your own URI
username = "neo4j"
password = "password"

# Initialize the Neo4j driver
driver = GraphDatabase.driver(bolt_uri, auth=(username, password))

# Initialize GraphDataScience instance
gds = GraphDataScience(bolt_uri, auth=(username, password))

# Step 9 - Create the bipartite graph for client similarity based on shared PIIs
query = """
MATCH (c:Client) 
WHERE c.secondPartyFraudRing IS NOT NULL
WITH collect(c) AS clients
MATCH (n) WHERE n:Email OR n:Phone OR n:SSN
WITH clients, collect(n) AS piis
WITH clients + piis AS nodes
MATCH (c:Client) -[:HAS_EMAIL | :HAS_PHONE | :HAS_SSN]->(p)
WHERE c.secondPartyFraudRing IS NOT NULL
WITH nodes, collect({source: c, target: p}) AS relationships
CALL gds.graph.project.cypher(
  'similarity',
  "UNWIND $nodes AS n RETURN id(n) AS id, labels(n) AS labels",
  "UNWIND $relationships AS r RETURN id(r['source']) AS source, id(r['target']) AS target, 'HAS_PII' AS type",
  {parameters: {nodes: nodes, relationships: relationships}}
)
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount
"""

# Execute the query to create the bipartite graph
with driver.session() as session:
    result = session.run(query)
    for record in result:
        print(f"Graph Name: {record['graphName']}")
        print(f"Node Count: {record['nodeCount']}")
        print(f"Relationship Count: {record['relationshipCount']}")

print("Bipartite graph 'similarity' created for client PII similarity.")


## Step 10

In [ ]:
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

# Connection details
bolt_uri = "bolt://localhost:7689"  # Adjust with your own URI
username = "neo4j"
password = "password"

# Initialize the Neo4j driver
driver = GraphDatabase.driver(bolt_uri, auth=(username, password))

# Initialize GraphDataScience instance
gds = GraphDataScience(bolt_uri, auth=(username, password))

# Cypher query to run nodeSimilarity algorithm
cypher_query_similarity = """
CALL gds.nodeSimilarity.mutate(
    'similarity',  // The name of the graph to mutate
    {
        mutateProperty: 'jaccardScore',
        mutateRelationshipType: 'SIMILAR_TO',
        topK: 15  // Top 15 most similar nodes
    }
)
"""

# Run the Cypher query via the Neo4j driver
with driver.session() as session:
    result = session.run(cypher_query_similarity)
    print("Successfully  computed similarity scores.")

## Step 11

In [ ]:
from neo4j import GraphDatabase

# Neo4j connection details
uri = "bolt://localhost:7689"  # Replace with your Neo4j URI
username = "neo4j"  # Replace with your Neo4j username
password = "password"  # Replace with your Neo4j password
driver = GraphDatabase.driver(uri, auth=(username, password))

# Function to write the relationships
def write_relationships(tx, graph_name):
    query = """
    CALL gds.graph.writeRelationship(
        $graph_name,
        'SIMILAR_TO',
        'jaccardScore'
    )
    YIELD relationshipsWritten
    RETURN relationshipsWritten
    """
    result = tx.run(query, graph_name=graph_name)
    return result.single()

# Run the node similarity algorithm in mutate mode
with driver.session() as session:
    result = session.execute_write(write_relationships, graph_name='similarity')
    print(f"Relationships Created: {result['relationshipsWritten']}")

## Step 12

In [ ]:
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

# Neo4j connection details
uri = "bolt://localhost:7689"  # Replace with your Neo4j URI
username = "neo4j"  # Replace with your username
password = "password"  # Replace with your password

# Create a Neo4j driver and GDS instance
driver = GraphDatabase.driver(uri, auth=(username, password))
gds = GraphDataScience(driver)

# Create a function to run the gds.degree.write procedure
def compute_degree_and_write(tx, graph_name):
    query = """
    CALL gds.degree.write(
        $graph_name,
        {
            nodeLabels: ['Client'],
            relationshipTypes: ['SIMILAR_TO'],
            relationshipWeightProperty: 'jaccardScore',
            writeProperty: 'secondPartyFraudScore'
        }
    )
    """
    tx.run(query, graph_name='similarity')

# Run the degree calculation and writing procedure in WRITE mode
with driver.session() as session:
    session.execute_write(compute_degree_and_write, 'similarity')
    print("Second party fraud property successfully written.")

## Step 13

In [ ]:
from neo4j import GraphDatabase

# Neo4j connection details
uri = "bolt://localhost:7689"  # Replace with your Neo4j URI
username = "neo4j"  # Replace with your Neo4j username
password = "password"  # Replace with your Neo4j password
driver = GraphDatabase.driver(uri, auth=(username, password))

# Function to label potential fraudsters based on degree of centrality
def label_potential_fraudster(tx, graph_name):
    query = """
    MATCH (c:Client)
    WHERE c.secondPartyFraudScore IS NOT NULL
    WITH percentileCont(c.secondPartyFraudScore, 0.95) AS threshold
        MATCH (c:Client)
        WHERE c.secondPartyFraudScore > threshold
        SET c.SecondPartyFraudster = true
    """
    tx.run(query, graph_name=graph_name)

# Execute the labeling within a write transaction
with driver.session() as session:
    session.execute_write(label_potential_fraudster, graph_name='similarity')
    print("Successfully labelled potential fraudsters with a high degree of centrality.")

## Step 14

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

# Neo4j connection details
uri = "bolt://localhost:7689"  # Replace with your Memgraph URI
username = "neo4j"  # Replace with your Neo4j username
password = "password"  # Replace with your Neo4j password
driver = GraphDatabase.driver(uri, auth=(username, password))

# Function to fetch fraudster clients and return as a Pandas DataFrame
def fetch_fraudster_clients(tx):
    query = """
    MATCH (c:Client)
    WHERE c.SecondPartyFraudster = true
    RETURN c.name AS clientName, ID(c) AS clientId
    """
    result = tx.run(query)
    
    # Collect results into a list of dictionaries to make it easier to import into a Pandas DataFrame
    clients = [{"clientName": record["clientName"], "clientId": record["clientId"]} for record in result]
    
    # Convert to Pandas DataFrame
    df = pd.DataFrame(clients)
    return df

# Execute the query within a read transaction
with driver.session() as session:
    fraudster_df = session.execute_read(fetch_fraudster_clients)
    
    # Display the results in the notebook using Pandas
    if not fraudster_df.empty:
        print("List of potential fraudsters:")
        display(fraudster_df)  # display() is used in Jupyter to render the DataFrame
    else:
        print("No potential fraudsters found.")